# Реализация алгоритма Advantage-Actor Critic (A2C) (вплоть до 10 баллов)

#### дедлайн задания: 3 мая, 23:59 GMT+3

# Работа выполнена: Макарчук Алексей Игоревич Б05-903в

В данной работе Вам предстоит реализовать алгоритм `Advantage Actor Critic`, обучаемый на батче из сред `Atari 2600`, работающих параллельно.

Для начала будут использованы обёртки сред, реализованные в файле `atari_wrappers.py`. Эти обёртки предварительно обрабатывают наблюдения (производят преобразования размера, цвета фрейма, взятия максимума между фреймами, пропускают часть фреймов и сводят несколько фреймов в один большой) и вознаграждения. Некоторые обёртки помогают автоматически перезапустить среду и присвоить переменной `done` значение `True` в случае смерти агента. Файл `env_batch.py` включает в себя реализацию класса `ParallelEnvBatch`, позволяющего запускать несколько сред параллельно. Для создания (инициализации) среды можно воспользоваться функцией `nature_dqn_env`. Обратите внимание, что в случае использования `PyTorch` (https://pytorch.org/) без `tensorboardX` (https://github.com/lanpa/tensorboardX) потребуется самостоятельно реализовать обёртку среды, которая будет логрировать **исходные** суммарные награды, которые *исходная* среда возвращает, и переопределить реализацию функции `nature_dqn_env`. То есть настоятельно рекомендуется применить `tensorboardX`.

Псевдокод алгоритма `Advantage Actor Critic (A2C)` приведён в **Разделе 5.2.5 (Алгоритм 20)** конспекта лекций: https://arxiv.org/pdf/2201.09746.pdf

Скрипты в данной работе используют Python версию библиотеки [OpenCV](https://pypi.org/project/opencv-python/). Для корректной работы ноутбука возможно понадобится доустановить библиотеку `gym`, запустив один из следующих наборов команд (пример приведён для Unix-подобных систем):
```
sudo apt-get install -y xvfb x11-utils ffmpeg python-opengl
pip install --upgrade pyglet pyvirtualdisplay opencv-python tqdm numpy gym[atari]==0.18.3
# wget https://bit.ly/2FMJP5K -O setup.py && bash setup.py # раскомментировать для Google Colab
```
или
```
sudo apt-get install -y xvfb x11-utils ffmpeg python-opengl
pip install --upgrade pyglet pyvirtualdisplay opencv-python tqdm numpy gym[all]==0.18.3
# wget https://bit.ly/2FMJP5K -O setup.py && bash setup.py # раскомментировать для Google Colab
```

In [ ]:
# XVFB будет запущен в случае исполнения на сервере
import os


if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

In [ ]:
import numpy as np
from atari_wrappers import nature_dqn_env


env = nature_dqn_env("SpaceInvadersNoFrameskip-v4", nenvs=8) # nenvs -- количество параллельно запущенных сред
                                                             # данный параметр можно варьировать для баланса
                                                             # производительность итерации/надёжность Монте-Карло оценок
                                                             # помните: при уменьшении nenvs, возможно, придётся
                                                             # увеличить количество итераций оптимизации
n_actions = env.action_space.spaces[0].n
obs = env.reset()
assert obs.shape == (8, 84, 84, 4)
assert obs.dtype == np.float64

В случае возникновения исключения `ROM is missing for space_invaders` выше рекомендуется установить ROM-ы согласно инструкции по адресу: https://github.com/openai/atari-py#roms.

Следующим шагом будет реализация модели, которая выводит логиты для категориального распределения на действия и оценку на значения $V$-функции ценности. Рекомендуется использовать архитектуру модели, представленной в публикации в журнале [Nature](https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf) со следующей модификацией: вместо одного выходного слоя нужно сделать два слоя, принимающих в качестве входа выход предшествующего скрытого слоя. **Обратите внимание**, данная модель отличается от модели, предложенной в домашней работе по DQN. Рекомендуется использовать ортогональную инициализацию с параметром $\sqrt{2}$ для ядер свёрток и инициализировать смещения нулями.

In [ ]:
import torch
import torch.optim as opt
from tqdm.notebook import tqdm


def conv2d_size_out(size, kernel_size, stride):
    """
    общий принцип использования данной функции:
    cur_layer_img_w = conv2d_size_out(cur_layer_img_w, kernel_size, stride)
    cur_layer_img_h = conv2d_size_out(cur_layer_img_h, kernel_size, stride)
    для вычисления размерности входа полносвязного слоя
    """
    return (size - (kernel_size - 1) - 1) // stride  + 1


class Flatten(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        return x.reshape(x.size(0), -1)


class NatureModel(torch.nn.Module):
    
    def __init__(self, bsize, height, width, nchannels, n_actions, device):
        super().__init__()
        self.device = device
        
        self.conv1 = torch.nn.Conv2d(in_channels=nchannels, out_channels=32, kernel_size=8, stride=4)
        self.relu1 = torch.nn.ReLU()
        cur_h, cur_w = conv2d_size_out(height, 8, 4), conv2d_size_out(width, 8, 4)
        self.conv2 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2)
        self.relu2 = torch.nn.ReLU()
        cur_h, cur_w = conv2d_size_out(cur_h, 4, 2), conv2d_size_out(cur_w, 4, 2)
        self.conv3 = torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1)
        self.relu3 = torch.nn.ReLU()
        cur_h, cur_w = conv2d_size_out(cur_h, 3, 1), conv2d_size_out(cur_w, 3, 1)
        self.flatten = Flatten()
        self.fc1 = torch.nn.Linear(in_features=64 * cur_h * cur_w, out_features=512)
        self.relu4 = torch.nn.ReLU()
        
        self.fc_logits = torch.nn.Linear(in_features=512, out_features=n_actions)
        self.fc_values = torch.nn.Linear(in_features=512, out_features=1)
        
        self._reset_parameters()
    
    def _reset_parameters(self):
        torch.nn.init.orthogonal_(self.conv1.weight.data, gain=np.sqrt(2))
        self.conv1.bias.data.fill_(0.0)
        
        torch.nn.init.orthogonal_(self.conv2.weight.data, gain=np.sqrt(2))
        self.conv2.bias.data.fill_(0.0)
        
        torch.nn.init.orthogonal_(self.conv3.weight.data, gain=np.sqrt(2))
        self.conv3.bias.data.fill_(0.0)
        
        torch.nn.init.orthogonal_(self.fc1.weight.data, gain=np.sqrt(2))
        self.fc1.bias.data.fill_(0.0)
        
        torch.nn.init.orthogonal_(self.fc_logits.weight.data, gain=np.sqrt(2))
        self.fc_logits.bias.data.fill_(0.0)
        
        torch.nn.init.orthogonal_(self.fc_values.weight.data, gain=np.sqrt(2))
        self.fc_values.bias.data.fill_(0.0)
    
    def forward(self, state):
        x = torch.tensor(state, device=self.device, dtype=torch.float).permute(0, 3, 1, 2)
        
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu4(x)
        
        logits = self.fc_logits(x)
        value = self.fc_values(x)
        
        x.cpu()
        del x
        
        return logits, value

Вам также потребуется определить и использовать политику, которая будет использовать модель выше. В то время как модель вычисляет логиты для всех действий сразу и оценку функции ценности, политика будет сэмплировать действия, а также будет вычислять их логарифм правдоподобия. Метод `Policy.act` должен возвращать словарь всех массивов, требуемых для взаимодействия со средой и обучения модели. Обратите внимание, что действия должны быть формата `Numpy.ndarray`, в то время как другие тензоры должны быть в формате, определяемом библиотекой глубокого обучения (`Torch.tensor`, например).

In [ ]:
from torch.distributions import Categorical


class Policy:
    def __init__(self, model):
        self.model = model

    def act(self, inputs):
        <Реализуйте политику посредством прямого прохода модели, сэмплирования действий и вычисления их логарифмов правдоподобия>
        # Должен возвращать dict с ключами ['actions', 'logits', 'log_probs', 'values'].
    
    def reset(self):
        pass

Далее требуется передать среду и политику в исполнитель `EnvRunner`, который собирает частичные траектории из среды. Класс `EnvRunner` уже реализован за Вас.

In [ ]:
from runners import EnvRunner

Данный исполнитель взаимодействует со средой заданное количество шагов и возвращает словарь, содержащий ключи:

* 'observations' 
* 'rewards' 
* 'resets'
* 'actions'
* и другие ключи, определённые в `Policy`

по каждому из этих ключей содержится Python `list` соответствующих результатов взаимодействий со средой указанной длины $T$ &mdash; размера частичной траектории.

Для того чтобы обучить часть модели, которая предсказывает ценности состояний, требуется вычислить целевые значения ценностей. В инстанцию класса `EnvRunner` можно подать при создании по аргументу `transforms` список вызываемых объектов ("функций"), которые последовательно будут применяться к частичным траекториям после сбора самих траекторий. Следовательно, требуется реализовать и использовать вызываемый (с определённым методом `__call__`) класс `ComputeValueTargets`. Формула для вычисления целевых значений ценности простая:

$$
\hat v(s_t) = \sum_{t'=0}^{T - 1}\gamma^{t'}r_{t+t'} + \gamma^T \hat{v}(s_{t+T}),
$$

Однако, не забудьте в реализации использовать `trajectory['resets']` флаги для проверки того, следует ли добавить целевые значения ценности на следующем шаге при вычислении целевых значений ценности на текущем шаге. У вас также имеется доступ к `trajectory['state']['latest_observation']` для получения последнего наблюдения в частичной траектории &mdash; $s_{t+T}$.

In [ ]:
class ComputeValueTargets:
    def __init__(self, gamma=0.99, device=torch.device('cuda:0')):
        self.gamma = gamma
        self.device = device

    def __call__(self, trajectory):
        # Данный метод должен модифицировать траекторию trajectory на месте через добавление
        # итеративно заполненного списка по ключу 'value_targets'.
        <Вычислите целевые значения ценности для данных частичных траекторий>

После вычисления целевых значений ценности требуется преобразовать списки результатов взаимодействия со средой в тензоры с первой компонентой размерности `batch_size`, равной призведению `T * nenvs`, то есть требуется свести в одну компоненту первые две компоненты размерности.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from collections import defaultdict


class MergeTimeBatch:
    """ Сращивает первые две оси, обычно отвечающие за время и за инстанцию среды, соответственно. """
    def __call__(self, trajectory):
        # Модификация траектории на месте. 
        <TODO: реализовать>

In [ ]:
seed = 617 # на своё усмотрение можно выбрать другой seed
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.benchmark = True


model = NatureModel(*obs.shape, n_actions, device)
model.to(device)
policy = Policy(model)
runner = EnvRunner(env, policy, nsteps=5, # nsteps -- длина частичной траектории
                                          # уменьшение nsteps может привести к вынужденному увеличению
                                          # количества итераций оптимизации
                   transforms=[ComputeValueTargets(device=device),
                               MergeTimeBatch()])

Настало время реализовать сам алгоритм Advantage-Actor Critic (A2C). Его псевдокод можно посмотреть в [конспектах лекций (Раздел 5.2.5)](https://arxiv.org/pdf/2201.09746.pdf), в публикции [Mnih et al. 2016](https://arxiv.org/abs/1602.01783) и в [лекции](https://www.youtube.com/watch?v=Tol_jw5hWnI&list=PLkFD6_40KJIxJMR-j5A1mkxK26gh_qg37&index=20) Сергея Левина.

In [ ]:
class A2C:
    def __init__(self,
               policy,
               optimizer,
               value_loss_coef=0.25,
               entropy_coef=0.01,
               max_grad_norm=0.5):
        self.policy = policy
        self.optimizer = optimizer
        self.value_loss_coef = value_loss_coef
        self.entropy_coef = entropy_coef
        self.max_grad_norm = max_grad_norm

    def policy_loss(self, trajectory):
        # Тут нужно вычислить Advantages. 
        <TODO: реализовать>

    def value_loss(self, trajectory):
        <TODO: реализовать>

    def loss(self, trajectory):
        <TODO: реализовать>

    def step(self, trajectory):
        <TODO: реализовать>

Теперь можно непосредственно обучить Вашу модель. С разумно подобранными гиперпараметрами обучение на одной GTX1080 на протяжении 10 миллионов шагов суммарно со всех батчированных сред (что переводится примерно в 5 часов работы) должно быть возможно достигнуть *среднюю исходную награду за 100 последних эпизодов* (значение переменной в `Tensorboard` по ключу `reward_mean_100`, усреднение берётся по 100 последним эпизодам в каждой среде в батче) **не меньше 600**. Это и будет считаться успешным результатом работы алгоритма `A2C`.

Вам так же, по возможности, рекомендуется отобразить данную величину относительно `runner.step_var` &mdash; количества взаимодействий со всеми средами. Также очень рекомендуется предоставить графики следующих показателей (полезно для отладки кода):
* [Коэффициент детерминации](https://en.wikipedia.org/wiki/Coefficient_of_determination) между целевыми значениями ценности и их предсказаниями
* Энтропия политики $\pi$
* Функция потерь ценности (Value loss)
* Функция потерь политики (Policy loss)
* Целевые значения ценности (Value targets)
* Предсказания значений ценности (Value predictions)
* Норма градиента
* Advantages
* Общая функция потерь (A2C loss)

В качестве оптимизатора рекомендуется взять метод [RMSProp](https://pytorch.org/docs/stable/generated/torch.optim.RMSprop.html) с [линейным убыванием шага](https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.LambdaLR.html), начиная с 7e-4 до 0, константой сглаживания (alpha в PyTorch и decay в TensorFlow), равной 0.99 и epsilon, равным 1e-5.

In [ ]:
a2c = <TODO: создать инстанцию класса A2C>

<TODO: реализовать цикл оптимизации политики согласно алгоритму A2C>

In [ ]:
test_env = nature_dqn_env("SpaceInvadersNoFrameskip-v4", nenvs=None, clip_reward=False, summaries=False)

In [ ]:
n_lives = 5


def evaluate(env, agent, n_games=1, greedy=False, t_max=10000):
    """ Играем n_games игр до конца. В случае жадной политики, выбираем действия как argmax(qvalues). Возвращаем среднюю награду. """
    rewards = []
    for _ in range(n_games):
        s = env.reset()
        reward = 0
        for _ in range(t_max):
            output = agent.act([s])
            action = output['logits'].argmax(dim=-1).item() if greedy else output['actions'][0]
            s, r, done, _ = env.step(action)
            reward += r
            if done:
                break

        rewards.append(reward)
    return np.mean(rewards)

In [ ]:
import gym.wrappers


env_monitor = gym.wrappers.Monitor(test_env, directory="videos", force=True)
sessions = [evaluate(env_monitor, policy, n_games=n_lives, greedy=True) for _ in range(10)]
env_monitor.close()

In [ ]:
env_monitor = gym.wrappers.Monitor(test_env, directory="stochasticvideos", force=True)
sessions = [evaluate(env_monitor, policy, n_games=n_lives, greedy=False) for _ in range(10)]
env_monitor.close()

In [ ]:
from IPython.display import HTML
from base64 import b64encode


video_names = list(
    filter(lambda s: s.endswith(".mp4"), os.listdir("./videos")))
mp4 = open("./videos/" + video_names[-1], "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(data_url))

In [ ]:
video_names = list(
    filter(lambda s: s.endswith(".mp4"), os.listdir("./stochasticvideos")))
mp4 = open("./stochasticvideos/" + video_names[-1], "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format(data_url))

# Информация об обучении

Прикрепление скриншотов графиков обучения модели в `Tensorboard` ниже является обязательным. Для доступа к `Tensorboard` запустите из командной строки в одной директории с данным ноутбуком следующую команду:
```
tensorboard --logdir logs --port 6006
```
В результате вывод в командную строку укажет, по какому адресу можно подсоединиться к инстанции `Tensorboard`, например, по адресу `http://localhost:6006/`. Оттуда можно и сделать скриншоты, демонстрирующие результаты обучения модели. Сами скриншоты с именем файла `image_name_x.png` для удобства лучше сохранить в директорию `./img`, откуда можно легко их прикреплять в `Markdown-клетках` ниже по команде со следующей конструкцией:
```
<img src=./img/image_name_x.png width=640>
```
Тут также требуется подписать изображения и дать небольшой комментарий по каждому скриншоту, что на нём описано.

**Внимание!** В случае перезапуска процедуры обучения модели рекомендуется удалить директорию `./logs` вместе с её содержимым перед непосредственным перезапуском, чтобы не испортить отображающиеся графики в `Tensorboard`.

**Совет.** При работе в Google Colab можно просто скачать директорию `./logs` и уже локально запустить `Tensorboard` для снятия скриншотов. Также можно обученного агента сохранить, скачать и локально на cpu запустить для записи роликов (для этого понадобится самостоятельно прописать код сохранения и загрузки модели в ноутбук из [файла](https://pytorch.org/tutorials/beginner/saving_loading_models.html)).

**Внимание!** Посылку для сдачи задания требуется оформить в виде `.zip` архива, в котором будут *данный ноутбук*, использованные для его работы *скрипты*, *директории* `./videos`, `./stochasticvideos`, `./logs` и `./img` с содержимым. Только так и не иначе!

__Вставьте в данную ячейку свой ответ, подкреплённый скриншотами__